# LSTMs for Human Activity Recognition

Human activity recognition using smartphones dataset and an LSTM RNN. Classifying the type of movement amongst six categories:
- WALKING,
- WALKING_UPSTAIRS,
- WALKING_DOWNSTAIRS,
- SITTING,
- STANDING,
- LAYING.

Compared to a classical approach, using a Recurrent Neural Networks (RNN) with Long Short-Term Memory cells (LSTMs) require no or almost no feature engineering. Data can be fed directly into the neural network who acts like a black box, modeling the problem correctly. Other research on the activity recognition dataset used mostly use a big amount of feature engineering, which is rather a signal processing approach combined with classical data science techniques. The approach here is rather very simple in terms of how much did the data was preprocessed. 

## Video dataset overview

Follow this link to see a video of the 6 activities recorded in the experiment with one of the participants:

<p align="center">
  <a href="http://www.youtube.com/watch?feature=player_embedded&v=XOEN9W05_4A
" target="_blank"><img src="http://img.youtube.com/vi/XOEN9W05_4A/0.jpg" 
alt="Video of the experiment" width="400" height="300" border="10" /></a>
  <a href="https://youtu.be/XOEN9W05_4A"><center>[Watch video]</center></a>
</p>

## Details about input data

I will be using an LSTM on the data to learn (as a cellphone attached on the waist) to recognise the type of activity that the user is doing. The dataset's description goes like this:

> The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. 

That said, I will use the almost raw data: only the gravity effect has been filtered out of the accelerometer  as a preprocessing step for another 3D feature as an input to help learning. 

## What is an RNN?

As explained in [this article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), an RNN takes many input vectors to process them and output other vectors. It can be roughly pictured like in the image below, imagining each rectangle has a vectorial depth and other special hidden quirks in the image below. **In our case, the "many to one" architecture is used**: we accept time series of feature vectors (one vector per time step) to convert them to a probability vector at the output for classification. Note that a "one to one" architecture would be a standard feedforward neural network. 

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" />

An LSTM is an improved RNN. It is more complex, but easier to train, avoiding what is called the vanishing gradient problem. 


## Results 

Scroll on! Nice visuals awaits. 

In [1]:
#loader class for loading and preprocessing the data sets
from loader import Loader
import time

# All Includes


import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics

import numpy as np
import tensorflow as tf
import keras
import os
import pickle
import pandas as pd

from keras.models import Sequential, Input, Model
from keras.layers.recurrent import LSTM
from keras import layers
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Reshape, Dropout, Conv1D, MaxPool1D
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from keras.layers.wrappers import TimeDistributed, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from keras import backend as K
from keras.regularizers import l2

np.random.seed(1337)
tf.set_random_seed(1339)

session_conf = tf.ConfigProto(log_device_placement=True)
# # Avoid full memory allocation on GPU
session_conf.gpu_options.allow_growth=True

sess = tf.InteractiveSession(config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


## Selecting a dataset and preparing it:

1) Run the appropriate loader from Loader  class <br>
2) Run the appropriate preprocessor (sliding window size , overlay) <br>
3) Load the pickled output file of the loader <br>

Could potentially be used in a loop for automated testing

In [2]:
load = Loader()


load.ward1_load()
load.ward1_preprocess(40,20)
path = "WARD1"
#load.opportunity_load()
#load.opportunity_preprocess(300,80)
#path = "OpportunityUCIDataset"
#load.daily_sports_load()
#load.daily_sports_preprocess(25,15)
#path = "Daily and sports activities"
#load.pamap2_load()
#load.pamap2_preprocess(270, 30)
#path = "PAMAP2"

#load.chiron_first_load()
#frame = load.chiron_first_preprocess(10, 5)
#path = "Chiron first"
#load.chiron_second_load()
#frame = load.chiron_second_preprocess(1, 0)
#path = "Chiron second"


#df = pickle.load(open('WARD1/processed_data.txt','rb')) # -> works
df = pickle.load(open('OpportunityUCIDataset/full_preprocessed_dataset.txt','rb')) # -> works
#df = pickle.load(open('Daily and sports activities/full_preprocessed.txt','rb')) #-> works
#df = pickle.load(open('PAMAP2/full_preprocessed.txt','rb'))

#df = pickle.load(open('Chiron first/Chiron_full_preprocessed.txt' ,'rb'))
#df = pickle.load(open('Chiron second/Chiron_full_preprocessed.txt' ,'rb'))




Downloading...
Dataset already downloaded. Did not download twice.

Extracting...
Dataset already extracted. Did not extract twice.

Successfully created additional folder: Results
--------------------------------------------------------
1 1 1
--------------------------------------------------------
1 1 2
--------------------------------------------------------
1 1 3
--------------------------------------------------------
1 1 4
--------------------------------------------------------
1 1 5
--------------------------------------------------------
1 2 1
--------------------------------------------------------
1 2 2
--------------------------------------------------------
1 2 3
--------------------------------------------------------
1 2 4
--------------------------------------------------------
1 2 5
--------------------------------------------------------
1 3 1
--------------------------------------------------------
1 3 2
--------------------------------------------------------
1 3 3

## Preparing the leave one out method:

Using methods that prepare the next leave one out division <br>

In [3]:

encoder = LabelEncoder()
encoder.fit(df.values[:,0])

classes = len(encoder.classes_)

def train_test(data, index):
    #training and test sets -> leave one out
    leave_out = df.index.isin([index])

    X_train = df[~leave_out].values[:,2:]
    y_train = df[~leave_out].values[:,0]

    X_test = df[leave_out].values[:,2:]
    y_test = df[leave_out].values[:,0]

    #--------------------------------------------------------------

    #normalization
    X_test = normalize(X_test)
    X_train = normalize(X_train)
    
    #--------------------------------------------------------------
    
    #reshaping to fit keras
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    return X_train, y_train, X_test, y_test

## Additionnal Parameters:

Here are some core parameter definitions for the training. 

The whole neural network's structure could be summarised by enumerating those parameters and the fact an LSTM is used. 

In [4]:
def additional_parameters(X_train, X_test):

    training_data_count = X_train.shape[0]  # 21522 training series (with 50% overlap between each serie)
    test_data_count = X_test.shape[0]  # 7794 testing series

    n_steps = X_train.shape[1]  # 128 timesteps per series
    n_input = len(X_train[0][0])  # 7 input parameters per timestep

    print("Steps: ",n_steps, "N imput",n_input)
    
    return training_data_count, test_data_count, n_steps, n_input

In [5]:
# Input Data 

# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = classes # Total classes (should go up, or should go down)

# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = 300  # Loop 300 times on the dataset
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
#print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test)) # -> takes too long
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [6]:
def one_hot(y_):
    #encoder = LabelEncoder()
    y_encoded = encoder.transform(y_.ravel())#fit_transform(y_.ravel())
    return np_utils.to_categorical(y_encoded, classes)

## Basic regression network

1) Get training and test data sets and calculate some additional parameters <br>
2) Use the additional parameters to build a model <br>
3) Train, test and evaluate the model <br>

In [7]:
#TODO -> add method calls and a for loop for leave one out

temp = 0 # for printing the current subject
num_epochs = 75 # for convenience
results_acc = np.array([[]]).reshape(num_epochs,0)
results_loss = np.array([[]]).reshape(num_epochs,0)
results_f1 = []


for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model

    input = Input(shape=(n_steps, n_input), name="input")
    x = Flatten()(input)
    x = Dense(n_hidden, activation="relu", name="fc1")(x)
    x = Dense(n_hidden, activation="relu", name="fc2")(x)
    x = Dense(n_classes, activation="softmax", name="prediction")(x)
    model = Model(input,x)
    #model.summary()

    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    #-------------------------------------------------------------
    #fit a model

    temp += 1
    print('Started fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    hist = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data=(X_test, y_test), verbose=0)
    
    
    #-------------------------------------------------------------
    #gather results
    
    #Accuracy and loss
    acc = np.array([hist.history['val_acc']]).reshape(len(hist.history['val_acc']),1)
    loss = np.array([hist.history['val_loss']]).reshape(len(hist.history['val_loss']),1)
    
    results_acc = np.concatenate([results_acc, acc], axis = 1)
    results_loss = np.concatenate([results_loss, loss], axis = 1)
    
    #Confusion matrix
    y_true = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    conf = pd.DataFrame(metrics.confusion_matrix(y_true, y_pred))
    conf.to_csv(path + "/Results/Basic_regression_Confidence_matrix_S" + str(int(index)) + ".csv", index=False)
    
    #F1 score
    results_f1.append(metrics.f1_score(y_true, y_pred, average='micro'))
    
    print('')
    print('Confusion matrix:')
    print(conf)
    print('F1 score ....... ' + str(results_f1[temp-1]))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(int(index)) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')


Steps:  72900 N imput 1
(296, 5)
(195, 5)
Started fitting model (leave out subject: 2) ....... 1

Confusion matrix:
    0   1  2  3  4
0  15  20  0  0  0
1   0  65  0  0  0
2   2  23  1  0  0
3   0  67  0  0  0
4   2   0  0  0  0
F1 score ....... 0.415384615385

Finished fitting model (leave out subject: 2) ....... 1
Total time: 44.214104413986206
------------------------------------------------------------
Steps:  72900 N imput 1
(283, 5)
(208, 5)
Started fitting model (leave out subject: 3) ....... 2

Confusion matrix:
    0   1  2   3  4
0   4   0  0   0  0
1  13  68  0  58  0
2  16  19  0   5  0
3   2   3  0  13  0
4   7   0  0   0  0
F1 score ....... 0.408653846154

Finished fitting model (leave out subject: 3) ....... 2
Total time: 44.3214693069458
------------------------------------------------------------
Steps:  72900 N imput 1
(403, 5)
(88, 5)
Started fitting model (leave out subject: 4) ....... 3

Confusion matrix:
    0   1   2  3  4
0  28  14  11  0  0
1   2  13   0  0  0

In [45]:
#print some results

print('Average accuracy ...... ' + str( np.sum(np.sum(results_acc, axis=0)/num_epochs)/temp) )
print('Average loss .......... ' + str( np.sum(np.sum(results_loss, axis=0)/num_epochs)/temp) )
print('Average F1 score ...... ' + str( sum(results_f1)/temp) )

Average accuracy ...... 0.39625097394
Average loss .......... 1.41461938063
Average F1 score ...... 0.427476689977


In [46]:
#add some labels and save the results to csv

#Accuracy and loss:
results_acc = pd.DataFrame(results_acc, columns = df.index.unique())
results_loss = pd.DataFrame(results_loss, columns = df.index.unique())

results_acc.to_csv(path + "/Results/Basic_regression_Accuracy.csv", index=False)
results_loss.to_csv(path + "/Results/Basic_regression_Loss.csv", index=False)

#F1 score:
results_f1 = np.array([results_f1])
results_f1 = pd.DataFrame(results_f1, columns = df.index.unique())
results_f1.to_csv(path + "/Results/Basic_regression_F1.csv", index=False)

## Code hasnt been tested past this point !!!!!!!!

## Basic LSTM network

In [ ]:

temp = 0
results = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(n_hidden, activation="relu", name="fc1")(input)
    x = LSTM(n_hidden, return_sequences=True, name='lstm_1')(x)
    x = LSTM(n_hidden, name='lstm_2')(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)
    model = Model(input,x)
    model.summary()
    
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model

    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 1, validation_data=(X_test, y_test), verbose=0) 
    results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    

Steps:  1000 N imput 1
(25277, 13)
(1456, 13)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1000, 1)           0         
_________________________________________________________________
fc1 (Dense)                  (None, 1000, 32)          64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 32)          8320      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
prediction (Dense)           (None, 13)                429       
Total params: 17,133
Trainable params: 17,133
Non-trainable params: 0
_________________________________________________________________
Started fitting model (leave out subject: 1.0) ....... 1
Train on 25277 samples, validate on

In [ ]:
avg = np.average(results, axis = 0)

print('Average loss .............. ' + str(avg[0]))
print('Average accuracy .......... ' + str(avg[1]))

## Basic bidirectional stacked LSTMs

In [53]:
temp = 0
results = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(n_hidden, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(n_hidden, return_sequences=True, name='lstm_1'))(x)
    x = Bidirectional(LSTM(n_hidden, name='lstm_2'))(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)
    model = Model(input,x)
    model.summary()
    
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 2, validation_data=(X_test, y_test), verbose=0)
    results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1000, 1)           0         
_________________________________________________________________
fc1 (Dense)                  (None, 1000, 32)          64        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 64)          16640     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                24832     
_________________________________________________________________
prediction (Dense)           (None, 2)                 130       
Total params: 41,666
Trainable params: 41,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
avg = np.average(results, axis = 0)

print('Average loss .............. ' + str(avg[0]))
print('Average accuracy .......... ' + str(avg[1]))

## 3x3 HAR stacked residual bidirectional LSTMs

In [56]:
learning_rate = 0.001
lambda_loss_amount = 0.005
clip_gradients = 15.0
n_layers_in_highway = 0
n_stacked_layers = 3
n_hidden = 32

In [57]:
temp = 0
results = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)


    ################  FIRST  ##################
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm1')(x)
    x = Dense(32, activation="relu", name="fc2")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm2')(x)
    shortcut = Dense(32, activation="relu", name="fc3")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn1')(x)

    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm3')(x)
    x = Dense(32, activation="relu", name="fc4")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm4')(x)
    shortcut = Dense(32, activation="relu", name="fc5")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn2')(x)

    x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm5')(x)
    x = Dense(32, activation="relu", name="fc6")(x)
    shortcut = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm6')(x)
    shortcut = Dense(32, activation="relu", name="fc7")(shortcut)
    x = layers.add([x, shortcut])
    # x = BatchNormalization(name='bn2')(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
    
        
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 20, validation_data=(X_test, y_test), verbose=0)
    #results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
    
    
    ############## Second ##########################

    
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm2')(x)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm3')(x)
    x = Dense(32, activation="relu", name="fc2")(x)
    # x = BatchNormalization(name='bn1')(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 200, validation_data=(X_test, y_test), verbose=0)
    #results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
    
    ################ Third ############################
    
    #-------------------------------------------------------------
    #make a model
    
    input = Input(shape=(n_steps, n_input), name="input")
    x = Dense(32, activation="relu", name="fc1")(input)
    x = Bidirectional(LSTM(32, return_sequences=True), name='bi_lstm1')(x)
    # x = Bidirectional(LSTM(16, return_sequences=True), name='bi_lstm2')(x)
    x = Dense(32, activation="relu", name="fc4")(x)

    x = Flatten()(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 200, validation_data=(X_test, y_test), verbose=0)
    results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 1000, 1)       0                                            
____________________________________________________________________________________________________
fc1 (Dense)                      (None, 1000, 32)      64          input[0][0]                      
____________________________________________________________________________________________________
bi_lstm1 (Bidirectional)         (None, 1000, 32)      6272        fc1[0][0]                        
____________________________________________________________________________________________________
fc2 (Dense)                      (None, 1000, 32)      1056        bi_lstm1[0][0]                   
___________________________________________________________________________________________

In [ ]:
avg = np.average(results, axis = 0)

print('Average loss .............. ' + str(avg[0]))
print('Average accuracy .......... ' + str(avg[1]))

## DeepConvLSTM
<br>
This Part of the code only does a summary of the model, the rest is missing

In [ ]:

input = Input(shape=(n_steps, n_input), name="input")
x = Dense(128, activation="relu", name="fc1")(input)
x = Bidirectional(LSTM(128, return_sequences=True), name='bi_lstm1')(x)
x = Bidirectional(LSTM(128, return_sequences=True), name='bi_lstm2')(x)
x = Dense(128, activation="relu", name="fc4")(x)

x = Flatten()(x)
x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

model = Model(input,x)
model.summary()


## Basic ConvLSTM

In [ ]:
temp = 0
results = []

for index in df.index.unique():

    #exception - subject does not contain all possible classes
    
    
    #-------------------------------------------------------------
    #get training and test sets

    X_train, y_train, X_test, y_test = train_test(df, index)

    training_data_count, test_data_count, n_steps, n_input = additional_parameters(X_train, X_test)
    training_iters = training_data_count * training_iters

    y_train = one_hot(y_train)
    y_test = one_hot(y_test)
    
    print(y_train.shape)
    print(y_test.shape)

    
    #-------------------------------------------------------------
    #make a model

    input = Input(shape=(n_steps, n_input), name="input")
    x = Conv1D(64, 5, activation='relu', name='conv1')(input)
    x = MaxPool1D()(x)
    x = Conv1D(64, 5, activation='relu', name='conv2')(x)
    x = MaxPool1D()(x)
    x = Flatten()(x)
    x = Dropout(.5)(x)
    x = Dense(n_classes, activation="softmax", activity_regularizer=l2(lambda_loss_amount), name="prediction")(x)

    model = Model(input,x)
    model.summary()
    
       
    #------------------------------------------------------------
    #choose an activation function

    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    
    #-------------------------------------------------------------
    #train a model
    
    temp += 1
    print('Started fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    
    t0 = time.time()
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs = 50, validation_data=(X_test, y_test), verbose=0)
    results.append(model.evaluate(X_test,y_test))
    print('')
    
    t1 = time.time()

    print('Finished fitting model (leave out subject: ' + str(index) + ') ....... ' + str(temp))
    print("Total time: " + str(t1-t0))
    print('------------------------------------------------------------')
    
    

Steps:  1125 N imput 1
(87780, 19)
(12540, 19)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1125, 1)           0         
_________________________________________________________________
conv1 (Conv1D)               (None, 1121, 64)          384       
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 560, 64)           0         
_________________________________________________________________


In [ ]:
avg = np.average(results, axis = 0)

print('Average loss .............. ' + str(avg[0]))
print('Average accuracy .......... ' + str(avg[1]))